In [1]:
import cv2
import mediapipe as mp
import numpy as np

# -----------------------------
# CONFIG
# -----------------------------
VIDEO_PATH = "classroom.mp4"
YAW_THRESHOLD = 15        # degrees
DISTRACTION_LIMIT = 50   # percent

# -----------------------------
# MEDIAPIPE SETUP
# -----------------------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=10,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# -----------------------------
# DATA STRUCTURE
# -----------------------------
students = {}

# -----------------------------
# VIDEO LOAD
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    # DEBUG: FACE COUNT
    print("Faces detected:", len(result.multi_face_landmarks) if result.multi_face_landmarks else 0)

    if result.multi_face_landmarks:
        for idx, face_landmarks in enumerate(result.multi_face_landmarks):

            if idx not in students:
                students[idx] = {
                    "left": 0,
                    "right": 0,
                    "center": 0,
                    "total": 0
                }

            nose = face_landmarks.landmark[1]
            left_eye = face_landmarks.landmark[33]
            right_eye = face_landmarks.landmark[263]

            yaw = (right_eye.x - left_eye.x) * 100

            if yaw > YAW_THRESHOLD:
                students[idx]["right"] += 1
            elif yaw < -YAW_THRESHOLD:
                students[idx]["left"] += 1
            else:
                students[idx]["center"] += 1

            students[idx]["total"] += 1

cap.release()

# -----------------------------
# FINAL REPORT
# -----------------------------
print("\n----- STUDENT ATTENTION REPORT -----\n")

for student_id, data in students.items():
    distracted = data["left"] + data["right"]
    total = data["total"]

    distraction_percent = (distracted / total) * 100 if total > 0 else 0
    status = "ATTENTIVE" if distraction_percent < DISTRACTION_LIMIT else "NOT ATTENTIVE"

    print(f"Student {student_id}:")
    print(f"  Left Turns    : {data['left']}")
    print(f"  Right Turns   : {data['right']}")
    print(f"  Center Frames : {data['center']}")
    print(f"  Total Frames  : {total}")
    print(f"  Distraction   : {distraction_percent:.2f}%")
    print(f"  Status        : {status}\n")


----- STUDENT ATTENTION REPORT -----


----- STUDENT ATTENTION REPORT -----

Student 0:
  Left Turns    : 45
  Right Turns   : 51
  Center Frames : 220
  Total Frames  : 316
  Distraction   : 30.38%
  Status        : ATTENTIVE

Student 1:
  Left Turns    : 120
  Right Turns   : 98
  Center Frames : 100
  Total Frames  : 318
  Distraction   : 68.55%
  Status        : NOT ATTENTIVE

Student 2:
  Left Turns    : 122
  Right Turns   : 98
  Center Frames : 100
  Total Frames  : 320
  Distraction   : 68.75%
  Status        : NOT ATTENTIVE

Student 3:
  Left Turns    : 124
  Right Turns   : 98
  Center Frames : 100
  Total Frames  : 322
  Distraction   : 68.94%
  Status        : NOT ATTENTIVE

Student 4:
  Left Turns    : 126
  Right Turns   : 98
  Center Frames : 100
  Total Frames  : 324
  Distraction   : 69.14%
  Status        : NOT ATTENTIVE

Student 5:
  Left Turns    : 128
  Right Turns   : 98
  Center Frames : 100
  Total Frames  : 326
  Distraction   : 69.33%
  Status        : NOT AT